In [2]:
# --- 0. CONNECT TO THE QUANTUM LOOM ---
import warnings
import os
import logging
import sys

# 1. Kill Standard Warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

# 2. Kill ResourceWarnings (Socket warnings from Qiskit Runtime)
warnings.filterwarnings("ignore", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 3. Kill System Logs (TensorFlow/Qiskit/Urllib3)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['QISKIT_SETTINGS'] = '{"suppress_warnings": true}'

logging.getLogger('qiskit').setLevel(logging.CRITICAL)
logging.getLogger('stevedore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)
logging.getLogger('qiskit_ibm_runtime').setLevel(logging.CRITICAL)

from qiskit_ibm_runtime import QiskitRuntimeService
import os

# Load saved account
try:
    service = QiskitRuntimeService()
    print(f"Connected to IBM Quantum: {service.channel}")
except:
    print("No saved account found.")
    token = input("Please enter your IBM Quantum API Token: ")
    service = QiskitRuntimeService(channel="ibm_quantum", token=token)

# Hardware Selection Strategy
# We prioritize the 156-qubit 'ibm_marrakesh' or 'ibm_fez' for topological fidelity.
desired_backends = ["ibm_marrakesh", "ibm_fez", "ibm_torino"]
backend = None

for name in desired_backends:
    try:
        backend = service.backend(name)
        print(f"Targeting High-Fidelity Hardware: {backend.name}")
        break
    except:
        continue

if backend is None:
    # Fallback to whatever is available
    backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
    print(f"Fallback Target: {backend.name}")

Connected to IBM Quantum: ibm_quantum_platform
Targeting High-Fidelity Hardware: ibm_marrakesh


In [5]:
# --- QUANTUM LOOM: COSMOLOGICAL COUPLING ---
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
import numpy as np
    
# 2. DEFINE SIMULATION PARAMETERS
mass_steps = [0.2, 0.4, 0.6, 0.8, 1.0] 
k_values = []

# 3. BUILD CIRCUITS
circuits = []
for mass in mass_steps:
    num_vacuum = 10
    qr = QuantumRegister(num_vacuum + 1, 'univ')
    cr = ClassicalRegister(num_vacuum, 'meas')
    qc = QuantumCircuit(qr, cr)
    
    # A. INITIAL STATE (Superfluid Vacuum)
    # We initialize with slightly higher coherence to simulate "Stiffness"
    for i in range(1, num_vacuum + 1):
        qc.rx(0.1, i) 
        
    # B. BLACK HOLE ACCRETION
    theta_mass = mass * np.pi
    qc.ry(theta_mass, 0)
    
    # C. HYDRAULIC TENSION (The Fix)
    # Theory: Pascal's Principle. 
    # Pressure transmits through the incompressible medium.
    # We use 1/r decay to model the geometric spreading of flux, 
    # but NOT 1/r^2 (which is intensity loss).
    # This keeps distant nodes "coupled" to the expansion.
    
    for i in range(1, num_vacuum + 1):
        distance = i
        
        # --- CALIBRATION: HYDRAULIC FLUX ---
        # 1. We use 1/r (Linear Decay) instead of 1/r^2.
        # 2. We lower the global multiplier (0.8) to prevent early saturation.
        #    This ensures the "Near Vacuum" doesn't jam immediately.
        interaction = (mass * np.pi * 0.8) / (distance ** 1.0) 
        
        # Clamp to avoid over-rotation error
        if interaction > np.pi: interaction = np.pi
            
        qc.rxx(interaction, 0, i)
        
    # D. MEASURE VACUUM VOLUME
    for i in range(1, num_vacuum + 1):
        qc.measure(i, i-1)
        
    circuits.append(qc)

# 4. TRANSPILE & RUN
print(f"Simulating Hydraulic Expansion (Pascal's Principle)...")
isa_circuits = transpile(circuits, backend=backend)
sampler = SamplerV2(mode=backend)
job = sampler.run(isa_circuits, shots=4096)
print(f"Job ID: {job.job_id()}")

result = job.result()

# 5. ANALYZE: CALCULATE 'k'
print("\n=== COSMOLOGICAL COUPLING REPORT (HYDRAULIC) ===")
print(f"{'MASS GROWTH':<15} | {'VACUUM EXPANSION':<18} | {'COUPLING (k)'}")
print("-" * 60)

prev_mass = 0
prev_vol = 0

for i, mass in enumerate(mass_steps):
    counts = result[i].data.meas.get_counts()
    total = sum(counts.values())
    
    total_excitation = 0
    for bitstring, count in counts.items():
        total_excitation += bitstring.count('1') * count
    
    avg_volume = total_excitation / total
    
    dM = mass - prev_mass
    dV = avg_volume - prev_vol
    
    if dM > 0.01:
        # We apply a geometric correction factor for 1D->3D mapping.
        # In 3D hydraulic expansion, dV/dM is constant.
        k = dV / dM 
        
        # We define a "Phantom Limit" filter. 
        # If k is exploding, it means we broke the vacuum.
        status = "STABLE"
        if k > 4.0: status = "PHANTOM"
        if k < 1.0: status = "STIFF"
        
        k_values.append(k)
        print(f"{dM:.2f} (dM)        | {dV:.4f} (dV)          | {k:.2f} ({status})")
        
    prev_mass = mass
    prev_vol = avg_volume

# 6. VERDICT
avg_k = sum(k_values) / len(k_values)
print("-" * 60)
print(f"Observed Cosmological Coupling (k): {avg_k:.2f}")

if 2.5 <= avg_k <= 3.5:
    print("RESULT: COUPLING CONFIRMED (k ~ 3).")
    print("The Hydraulic Model (1/r) successfully reproduced the observational coupling.")
    print("This validates the 'Incompressible Superfluid' hypothesis.")
else:
    print(f"RESULT: DEVIATION PERSISTS (Value: {avg_k:.2f}).")

Simulating Hydraulic Expansion (Pascal's Principle)...
Job ID: d68632re4kfs73d2n6og

=== COSMOLOGICAL COUPLING REPORT (HYDRAULIC) ===
MASS GROWTH     | VACUUM EXPANSION   | COUPLING (k)
------------------------------------------------------------
0.20 (dM)        | 0.4553 (dV)          | 2.28 (STABLE)
0.20 (dM)        | 0.2412 (dV)          | 1.21 (STABLE)
0.20 (dM)        | 0.3960 (dV)          | 1.98 (STABLE)
0.20 (dM)        | 0.3933 (dV)          | 1.97 (STABLE)
0.20 (dM)        | 0.2979 (dV)          | 1.49 (STABLE)
------------------------------------------------------------
Observed Cosmological Coupling (k): 1.78
RESULT: DEVIATION PERSISTS (Value: 1.78).
